In [30]:
import pandas as pd

from power import panel_mde
from tabulate import tabulate

In [31]:
panel_in = "../../../data/final/model_data.parquet"
mde_out = "../../../reports/replication/mde.md"

In [32]:
panel = pd.read_parquet(panel_in)

In [33]:
mde_stats = panel_mde(panel, ['transit'], 'id', 'rides', 'UCMP == 1')

In [34]:
# Format for presentation.
mde_stats.index.set_names(['Transit'], inplace=True)
mde_stats['Mean Rides per Unit per Day (SD)'] = mde_stats['ar_mean'].apply("{:.1f}".format) + \
                                                mde_stats['ar_std'].apply(" ({:.1f})".format)
mde_stats['Minimum Detectable Change'] = "+" + mde_stats['mdc'].apply("{:.0%}".format)
mde_stats['Minimum Additional Rides'] = mde_stats['mds'].apply("{:.0f}".format)
mde_stats.drop(columns=['ar_mean','ar_std','mdc','mds',
                        'geo_mean','exp(geo_mean)', 'geo_std',
                        'se', 'exp(se)', 'mde', 'n'], inplace=True)
mde_stats

,Mean Rides per Unit per Day (SD),Minimum Detectable Change,Minimum Additional Rides
Transit,,,
bike,50.3 (17.5),+151%,10235
train,1482.0 (392.7),+227%,58655
uber,753.1 (583.5),+142%,55036


In [35]:

# Note: For future reference if you need multi-header columns:
# The best option is to tabualte with tablefmt=html, open the file and
# added a new <tr> in the table header, fixed the column name labels, 
# and used the <th colspan="2"> attribute to make the spanning columns.

with open(mde_out,"w") as f:
    tbl = tabulate(mde_stats.reset_index(), 
                   headers=mde_stats.reset_index().columns, 
                   showindex=False,
                   tablefmt='github')
    f.write(tbl)
    print(tbl)

!cp {mde_out} ../../../../eric-mc2-cv/static/uploads/

| Transit   | Mean Rides per Unit per Day (SD)   | Minimum Detectable Change   |   Minimum Additional Rides |
|-----------|------------------------------------|-----------------------------|----------------------------|
| bike      | 50.3 (17.5)                        | +151%                       |                      10235 |
| train     | 1482.0 (392.7)                     | +227%                       |                      58655 |
| uber      | 753.1 (583.5)                      | +142%                       |                      55036 |
